# Scattering resonances for a circular metamaterial cavity

The codes below are associated to the article:

- C. Carvalho and Z. Moitier, _Asymptotics for metamaterial cavities and their effect on scattering_ [[arXiv](https://arxiv.org/abs/2010.07583), [HAL](https://hal.archives-ouvertes.fr/hal-02965993)]

We present computations related to Section 3 and Appendix C of the manuscript.

## Zoïs Moitier, Camille Carvalho (2021)
            
_Karlsruhe Institute of Technology, Germany_

_University of California Merced, USA_

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets
from matplotlib.colors import Normalize, SymLogNorm

import src

%matplotlib inline

## Internal functions

In [2]:
def parse_data(data):
    return (data[:, 0].astype(int), np.power(data[:, 1] + 1j * data[:, 2], 2))

In [3]:
def plot_scatter(ax, m, λ, M, lw, marker, zorder, label):
    ind = np.where(np.less_equal(m, M))

    sc = ax.scatter(
        np.real(λ[ind]),
        np.imag(λ[ind]),
        s=15,
        lw=lw,
        c=m[ind],
        marker=marker,
        zorder=zorder,
        vmin=0,
        vmax=M,
        label=label,
    )
    return sc

In [4]:
def plot_λ(ε, M, ax, xlim=None, ylim=None):
    η = np.sqrt(-ε)

    data = np.load(f"data/eps_{ε}.npz")
    m_out, λ_out = parse_data(data["outer"])
    m_inn, λ_inn = parse_data(data["inner"])
    m_pla, λ_pla = parse_data(data["plasmon"])

    _ = plot_scatter(ax, m_out, λ_out, M, 0, "v", 2, r"$\mathcal{R}_{\mathsf{out}}$")
    sc_inn = plot_scatter(
        ax, m_inn, λ_inn, M, 0.1, ".", 3, r"$\mathcal{R}_{\mathsf{inn}}$"
    )
    _ = plot_scatter(ax, m_pla, λ_pla, M, 0.5, "+", 4, r"$\mathcal{R}_{\mathsf{pla}}$")

    if xlim is not None:
        ax.set_xlim(*xlim)
    if ylim is not None:
        ax.set_ylim(*ylim)

    ax.set_xlabel(r"$\Re(\ell^2)$")
    ax.set_ylabel(r"$\Im(\ell^2)$")

    ax.grid(True, zorder=1)
    ax.legend(
        loc="lower center",
        bbox_to_anchor=(0.5, 1.025),
        ncol=3,
        borderaxespad=0.0,
        handlelength=0,
        markerscale=2,
    )
    ax.set_title(f"ε = {ε}", y=1.125)

    return sc_inn

## Scattering resonances

In [5]:
εcav = [-1.5, -1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7]


@interact
def interact_plot_resonances(
    M: (1, 64, 6) = 64,
    ε1=widgets.Dropdown(options=εcav, value=-1.1, description="ε left"),
    ε2=widgets.Dropdown(options=εcav, value=-0.9, description="ε right"),
    xmin="insert number",
    xmax="insert number",
    ymin="insert number",
    ymax="insert number",
):
    if src.is_number(xmin) and src.is_number(xmax):
        xlim = (float(xmin), float(xmax))
    else:
        xlim = None

    if src.is_number(ymin) and src.is_number(ymax):
        ylim = (float(ymin), float(ymax))
    else:
        ylim = None

    plt.rcParams.update(src.set_rcParams(font_size=15))
    fig, ax = plt.subplots(
        ncols=2,
        figsize=src.set_size(frac_width=2, frac_height=0.5),
        constrained_layout=True,
    )

    sc1 = plot_λ(ε1, M, ax[0], xlim=xlim, ylim=ylim)
    sc2 = plot_λ(ε2, M, ax[1], xlim=ylim, ylim=ylim)

    cbar = fig.colorbar(sc1, ax=ax[0])
    cbar.set_label(r"$m$")

    cbar = fig.colorbar(sc2, ax=ax[1])
    cbar.set_label(r"$m$")

    return None

interactive(children=(IntSlider(value=64, description='M', max=64, min=1, step=6), Dropdown(description='ε lef…